In [1]:
import bson
from tqdm import tqdm
from collections import Counter

In [2]:
bson_path = "/data/data/train_example.bson"
n = 82

In [3]:
data = bson.decode_file_iter(open(bson_path, 'rb'))

category_ids = []
for c, d in tqdm(enumerate(data), total=n):
    category_ids.append(d["category_id"])

100%|██████████| 82/82 [00:00<00:00, 16572.68it/s]


In [4]:
categories_counts = Counter(category_ids).most_common()
categories_counts[:10]

[(1000010653, 36),
 (1000010667, 4),
 (1000004079, 4),
 (1000004141, 2),
 (1000010683, 2),
 (1000015309, 2),
 (1000018290, 2),
 (1000018294, 2),
 (1000018306, 1),
 (1000013922, 1)]

In [6]:
len(categories_counts)

36

In [8]:
def one_hot_label_encoder(csv_path):
    _category_labels = pd.read_csv(csv_path, dtype={'category_id': np.str})
    _lb = preprocessing.LabelBinarizer()
    _lb.fit(_category_labels['category_id'])

    def find_max_idx(lb_vec):
        _lb_vector = lb_vec.reshape(-1).tolist()
        return _lb_vector.index(max(_lb_vector))

    def encode(lbs_str):
        _lbs_vector = np.asarray(_lb.transform(lbs_str), dtype=np.float32)
        return np.apply_along_axis(find_max_idx, 1, _lbs_vector)

    def decode(one_hots):
        # _lbs_vector = label_vector(one_hots)
        return _lb.inverse_transform(np.array(one_hots))

    return encode, decode

one_hot_encoder, _ = one_hot_label_encoder(csv_path="/data/data/category_names.csv")

def cal_weight(t):
    weight = float(79640) / t[1]
    weight = weight if weight < 20 else 20
    return (one_hot_encoder([str(t[0])])[0], round(weight,4))

In [9]:
class_weight = map(cal_weight, categories_counts)

In [11]:
class_weight[-10:]

[(1000010653, 20.0),
 (1000010667, 20.0),
 (1000004079, 20.0),
 (1000004141, 20.0),
 (1000010683, 20.0),
 (1000015309, 20.0),
 (1000018290, 20.0),
 (1000018294, 20.0),
 (1000018306, 20.0),
 (1000013922, 20.0),
 (1000012993, 20.0),
 (1000003977, 20.0),
 (1000012558, 20.0),
 (1000010641, 20.0),
 (1000010645, 20.0),
 (1000005910, 20.0),
 (1000010647, 20.0),
 (1000011423, 20.0),
 (1000005796, 20.0),
 (1000015539, 20.0),
 (1000014396, 20.0),
 (1000012989, 20.0),
 (1000007361, 20.0),
 (1000001859, 20.0),
 (1000003787, 20.0),
 (1000010061, 20.0),
 (1000010961, 20.0),
 (1000010706, 20.0),
 (1000014287, 20.0),
 (1000010461, 20.0),
 (1000007138, 20.0),
 (1000014053, 20.0),
 (1000005990, 20.0),
 (1000005744, 20.0),
 (1000004085, 20.0),
 (1000003191, 20.0)]